In [1]:
import math
from typing import List
import sys
import os
from pathlib import Path
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.discriminant_analysis import StandardScaler
from copy import deepcopy
import torch.optim as optim
import optuna
from optuna.trial import TrialState
import statistics
from sklearn.model_selection import KFold
import time
from mlp import MLP, CustomDataset
from utils import calculate_metric

In [2]:
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))

In [19]:
DATA_FOLDER = "../data"
TRAIN_FEATURES = DATA_FOLDER + '/train_features2.xlsx'
TRAIN_LABELS = DATA_FOLDER + "/train_labels2.xlsx"
TEST_FEATURES = DATA_FOLDER + "/test_features2.xlsx"
TEST_LABELS = DATA_FOLDER + "/test_labels2.xlsx"

OUTPUT_PATH = '../output/mlp'

TRAINING_OUTPUT_FILE = '../output/train_predictions.xlsx'
TEST_OUTPUT_FILE = '../output/test_predictions.xlsx'
SHEET_NAME = "mlp"
OOF_PREDICTIONS_FILE = '../output/oof_predictions.xlsx'

MODEL_PATH = OUTPUT_PATH + '/best_mlp_model.pth'
BATCH_SIZE = 1

DEVICE = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
DEVICE_LIST = [1, 3, 2, 0]

Path(OUTPUT_PATH).mkdir(parents=True, exist_ok=True)

In [18]:
DEVICE

device(type='cuda', index=0)

In [5]:
train_feature_df = pd.read_excel(TRAIN_FEATURES)
train_label_df = pd.read_excel(TRAIN_LABELS)
test_feature_df = pd.read_excel(TEST_FEATURES)
test_label_df = pd.read_excel(TEST_LABELS)

In [6]:
feature_list = train_feature_df.columns
category_feature_key = ['currency', 'seniorioty_adj', 'domicile_country',	'exchange_country',	'Industry_sector',	'Industry_group',	'Industry_subgroup', 'event_type',
                        'event_type_subcategory_sum']
category_features = [i for i in feature_list if any(sub in i for sub in category_feature_key)]
non_category_features = [i for i in feature_list if i not in category_features]

print(len(category_features))
print(len(non_category_features))

162
155


In [7]:
# Normalize the data
# Prepare the ColumnTransformer
scaler = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), non_category_features)   # StandardScaler()
    ],
    remainder='passthrough'  # Leave categorical features untouched
)



TUNING PARAMETERS

In [8]:
EPOCHS = 50

In [9]:
def objective(trial):
    # Define hyperparameters
    num_blocks = trial.suggest_int('num_blocks', 1, 5)
    
    out_features_list = []
    for i in range(num_blocks):
        out_features = trial.suggest_int(f'out_features_{i}', 10, 128)
        out_features_list.append(out_features)

    p = trial.suggest_float("dropout_l{}".format(i), 0, 0.5)
    activation_name = trial.suggest_categorical(f'activation_{i}', ['ReLU', 'Tanh', 'LeakyReLU'])
    negative_slope = 0.01
    if activation_name == 'LeakyReLU':
        negative_slope = trial.suggest_float(f"negative_slope_{i}", 0.01, 1)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-9, 1e-1, log=True)

    # Define cross-validation setup
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    val_losses = []

    for train_idx, val_idx in kf.split(train_feature_df):
        # Create training and validation datasets for the current fold
        X_train_fold, X_val_fold = train_feature_df.iloc[train_idx], train_feature_df.iloc[val_idx]
        y_train_fold, y_val_fold = train_label_df.iloc[train_idx], train_label_df.iloc[val_idx]
        
        # scaling features
        X_train_fold = scaler.fit_transform(X_train_fold)
        X_val_fold = scaler.transform(X_val_fold)
            
        # Initialize the model for this fold
        model = MLP(d_in=X_train_fold.shape[1], d_layers=out_features_list, dropout=p, d_out=1, activation_name=activation_name, negative_slope=negative_slope)
        model = nn.DataParallel(model, device_ids = DEVICE_LIST)
        model.to(DEVICE)
        
        # define optimizer
        if optimizer_name == "Adam":
         optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr, weight_decay=weight_decay)
        else:
            momentum = trial.suggest_float("momentum", 1e-9, 0.95, log=True)
            optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr, weight_decay=weight_decay, momentum=momentum)
        
        # Define the loss function and optimizer
        criterion = nn.MSELoss()
        
        # Prepare DataLoader for training
        train_dataset = CustomDataset(X_train_fold, y_train_fold.to_numpy())
        val_dataset = CustomDataset(X_val_fold, y_val_fold.to_numpy())
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
            
        # Training of the model.
        model.train()
        for epoch in range(EPOCHS):
            for batch_idx, (data, target) in enumerate(train_loader):
                data, target = data.to(DEVICE), target.to(DEVICE)

                optimizer.zero_grad()
                output = model(data)
                # print("shape", output.shape, target.shape)
                loss = criterion(output, target)
                loss.backward()
                optimizer.step()

        # Validation of the model.
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(val_loader):
                data, target = data.to(DEVICE), target.to(DEVICE)
                output = model(data)
                val_loss = criterion(output, target).item()
                val_losses.append(val_loss**0.5) #rmse

        trial.report(val_loss, epoch)

    # Return the average validation loss across all folds
    # return torch.stack(val_losses).mean()
    return np.mean(val_losses)


In [19]:
# 3. Create a study object and optimize the objective function.
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100, timeout=600)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


[I 2024-08-25 15:54:44,673] A new study created in memory with name: no-name-eb8841d3-adf5-46ea-926a-11e0f35b94ac
/home/nguyenhoa/miniconda3/envs/.henv/lib/python3.12/site-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 49 is already reported.
  warnings.warn(
/home/nguyenhoa/miniconda3/envs/.henv/lib/python3.12/site-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 49 is already reported.
  warnings.warn(
/home/nguyenhoa/miniconda3/envs/.henv/lib/python3.12/site-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 49 is already reported.
  warnings.warn(
/home/nguyenhoa/miniconda3/envs/.henv/lib/python3.12/site-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 49 is already reported.
  warnings.warn(
[I 2024-08-25 16:01:49,560] Trial 0 finished with value: 2432.6910608637604 and parameters

Study statistics: 
  Number of finished trials:  2
  Number of pruned trials:  0
  Number of complete trials:  2
Best trial:
  Value:  0.26464127320230924
  Params: 
    num_blocks: 5
    out_features_0: 125
    out_features_1: 97
    out_features_2: 35
    out_features_3: 43
    out_features_4: 33
    dropout_l4: 0.027688994176150394
    activation_4: LeakyReLU
    negative_slope_4: 0.5381973959626298
    optimizer: SGD
    lr: 0.00030180928303074906
    weight_decay: 3.347848174151389e-05
    momentum: 0.00030930595636861923


In [ ]:
trial.params

{'num_blocks': 4,
 'out_features_0': 68,
 'out_features_1': 18,
 'out_features_2': 43,
 'out_features_3': 30,
 'dropout_l3': 0.36051593533006826,
 'activation_3': 'Tanh',
 'optimizer': 'Adam',
 'lr': 0.0012567816952842909,
 'weight_decay': 2.0076983019785407e-06}

In [10]:
# best_params = trial.params
best_params = {'num_blocks': 4,
 'out_features_0': 68,
 'out_features_1': 18,
 'out_features_2': 43,
 'out_features_3': 30,
 'dropout_l3': 0.36051593533006826,
 'activation_3': 'Tanh',
 'optimizer': 'Adam',
 'lr': 0.0012567816952842909,
 'weight_decay': 2.0076983019785407e-06}

In [11]:
MODEL_CONFIG = {"linear": [], "dropout": 0, "optimizer": {}, "negative_slope": 0.01, "activation_name": ""}

for key, value in best_params.items():
    if "out_features" in key:
        MODEL_CONFIG["linear"].append(value)
    elif "dropout" in key:
        MODEL_CONFIG["dropout"] = value
    elif "negative_slope" in key:
        MODEL_CONFIG["negative_slope"] = value
    elif "activation" in key:
        MODEL_CONFIG["activation_name"] = value
    elif "num_blocks" in key:
        continue
    else:
        MODEL_CONFIG["optimizer"][key] = value

In [12]:
MODEL_CONFIG

{'linear': [68, 18, 43, 30],
 'dropout': 0.36051593533006826,
 'optimizer': {'optimizer': 'Adam',
  'lr': 0.0012567816952842909,
  'weight_decay': 2.0076983019785407e-06},
 'negative_slope': 0.01,
 'activation_name': 'Tanh'}

RUNNING THE MODEL

In [13]:
# Transform training data
train_features = scaler.fit_transform(train_feature_df)
test_features = scaler.transform(test_feature_df)

new_feature_list = non_category_features + category_features

In [14]:
# Create dataset instances
train_dataset = CustomDataset(train_features, train_label_df.to_numpy())
test_dataset = CustomDataset(test_features, test_label_df.to_numpy())

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=test_feature_df.shape[0], shuffle=False)

In [23]:
# model = MLP(d_in=train_features.shape[1], d_layers=MODEL_CONFIG["linear"], dropout=MODEL_CONFIG["dropout"], d_out=1).to(DEVICE)
model = MLP(d_in=train_features.shape[1], d_layers=MODEL_CONFIG["linear"], dropout=MODEL_CONFIG["dropout"], d_out=1, activation_name=MODEL_CONFIG["activation_name"], negative_slope=MODEL_CONFIG['negative_slope'])
model

MLP(
  (linear_layers): ModuleList(
    (0): Linear(in_features=317, out_features=68, bias=True)
    (1): Linear(in_features=68, out_features=18, bias=True)
    (2): Linear(in_features=18, out_features=43, bias=True)
    (3): Linear(in_features=43, out_features=30, bias=True)
  )
  (dropout_layer): Dropout(p=0.36051593533006826, inplace=False)
  (head): Linear(in_features=30, out_features=1, bias=True)
  (activation_layer): Tanh()
)

In [21]:
# empty cache first
torch.cuda.empty_cache()

In [22]:
model = nn.DataParallel(model, device_ids = DEVICE_LIST)
model.to(DEVICE)

DataParallel(
  (module): MLP(
    (linear_layers): ModuleList(
      (0): Linear(in_features=317, out_features=68, bias=True)
      (1): Linear(in_features=68, out_features=18, bias=True)
      (2): Linear(in_features=18, out_features=43, bias=True)
      (3): Linear(in_features=43, out_features=30, bias=True)
    )
    (dropout_layer): Dropout(p=0.36051593533006826, inplace=False)
    (head): Linear(in_features=30, out_features=1, bias=True)
    (activation_layer): Tanh()
  )
)

In [21]:
# define optimizer
optim_config = deepcopy(MODEL_CONFIG["optimizer"])
del optim_config["optimizer"]

optimizer = getattr(optim, MODEL_CONFIG["optimizer"]["optimizer"])(model.parameters(), **optim_config)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.0012567816952842909
    maximize: False
    weight_decay: 2.0076983019785407e-06
)

In [30]:
EPOCH = 500
criterion = nn.MSELoss()
start_time = time.time()

for ep in tqdm(range(EPOCH)):

    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        
        loss.backward()

        optimizer.step()

        # print statistics
        running_loss += loss.item() * inputs.size(0)
        if i % 100 == 99:    # print every 100 mini-batches
            print(f'[{ep + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}')


train_loss = running_loss  / len(train_loader.dataset)
print(f'Epoch [{ep+1}], Train Loss: {train_loss:.4f}')

# print out training time
elapsed_time = time.time() - start_time
print(f"Training time: {elapsed_time:.3f} seconds")

# save the best model
torch.save(model.state_dict(), MODEL_PATH)

  0%|          | 0/500 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


[1,   100] loss: 0.055
[1,   200] loss: 0.098
[1,   300] loss: 0.139
[1,   400] loss: 0.190
[1,   500] loss: 0.244
[1,   600] loss: 0.280
[1,   700] loss: 0.314
[1,   800] loss: 0.358
[1,   900] loss: 0.414
[1,  1000] loss: 0.470
[1,  1100] loss: 0.535
[1,  1200] loss: 0.574


  0%|          | 1/500 [00:02<23:36,  2.84s/it]

[2,   100] loss: 0.043
[2,   200] loss: 0.080
[2,   300] loss: 0.130
[2,   400] loss: 0.161
[2,   500] loss: 0.224
[2,   600] loss: 0.275
[2,   700] loss: 0.342
[2,   800] loss: 0.396
[2,   900] loss: 0.446
[2,  1000] loss: 0.489
[2,  1100] loss: 0.553
[2,  1200] loss: 0.593


  0%|          | 2/500 [00:06<25:54,  3.12s/it]

...

[499,   100] loss: 0.049
[499,   200] loss: 0.087
[499,   300] loss: 0.151
[499,   400] loss: 0.198
[499,   500] loss: 0.248
[499,   600] loss: 0.306
[499,   700] loss: 0.341
[499,   800] loss: 0.404
[499,   900] loss: 0.453
[499,  1000] loss: 0.498
[499,  1100] loss: 0.551
[499,  1200] loss: 0.610


100%|█████████▉| 499/500 [35:19<00:03,  3.99s/it]

[500,   100] loss: 0.046
[500,   200] loss: 0.093
[500,   300] loss: 0.139
[500,   400] loss: 0.197
[500,   500] loss: 0.261
[500,   600] loss: 0.315
[500,   700] loss: 0.361
[500,   800] loss: 0.424
[500,   900] loss: 0.471
[500,  1000] loss: 0.514
[500,  1100] loss: 0.553
[500,  1200] loss: 0.598


100%|██████████| 500/500 [35:23<00:00,  4.25s/it]

Epoch [500], Train Loss: 0.0494
Training time: 2123.254 seconds


In [39]:
torch.save(model.state_dict(), MODEL_PATH)

In [31]:
torch.cuda.empty_cache()

In [46]:
# Testing phase
model.eval()
# Lists to store predictions and targets
all_predictions = []
all_targets = []

with torch.no_grad():
    for inputs, labels in train_loader:
        # inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        
        if isinstance(model, nn.DataParallel):
            model = model.module  # Unwrap from DataParallel
        model = model.to('cpu')
        
        outputs = model(inputs)
        # Collect predictions and targets
        all_predictions.extend(outputs.cpu().numpy())
        all_targets.extend(labels.cpu().numpy())

# Convert to numpy arrays
y_pred = np.array(all_predictions)
y_true = np.array(all_targets)

# save metrics
mae, mape, rmse, rsqr = calculate_metric(y_pred, y_true)
print(f"Training average mean absolute error: {mae}")
print(f"Training average mean absolute percentage error: {mape}")
print(f"Training average root mean squared error: {rmse}")
print(f"Training average R2: {rsqr}")

Training average mean absolute error: 0.33150047063827515
Training average mean absolute percentage error: 4655.685043334961
Training average root mean squared error: 0.19017133549491563
Training average R2: 0.6666419506072998


In [47]:
# Testing phase
model.eval()
all_predictions = []
all_targets = []

with torch.no_grad():
    for inputs, labels in test_loader:
        # inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        
        if isinstance(model, nn.DataParallel):
            model = model.module  # Unwrap from DataParallel
        model = model.to('cpu')
        
        outputs = model(inputs)
        # Collect predictions and targets
        all_predictions.extend(outputs.cpu().numpy())
        all_targets.extend(labels.cpu().numpy())

# Convert to numpy arrays
y_test_pred = np.array(all_predictions)
y_test_true = np.array(all_targets)

# save metrics
mae, mape, rmse, rsqr = calculate_metric(y_test_pred, y_test_true)
print(f"Test average mean absolute error: {mae}")
print(f"Test average mean absolute percentage error: {mape}")
print(f"Test average root mean squared error: {rmse}")
print(f"Test average R2: {rsqr}")

Test average mean absolute error: 0.337220162153244
Test average mean absolute percentage error: 384.48150157928467
Test average root mean squared error: 0.26011602531800193
Test average R2: 0.4093068838119507


In [49]:
# save train predictions
train_predictions_df = pd.DataFrame({"predictions": y_pred.ravel()})
with pd.ExcelWriter(TRAINING_OUTPUT_FILE, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    # Write the new DataFrame to a new sheet
    train_predictions_df.to_excel(writer, sheet_name=SHEET_NAME, index=False)
    
# save test predictions
test_predictions_df = pd.DataFrame({"predictions": y_test_pred.ravel()})
with pd.ExcelWriter(TEST_OUTPUT_FILE, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    # Write the new DataFrame to a new sheet
    test_predictions_df.to_excel(writer, sheet_name=SHEET_NAME, index=False)


In [24]:
# let’s load back in our saved model
# model = MLP()
# model.load_state_dict(torch.load(MODEL_PATH))

/var/folders/gc/mln0kg89025flmz65rqz4qn40000gn/T/ipykernel_79046/4150849819.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_PATH))

<All keys matched successfully>

RUN 5-fold

In [15]:
features = pd.concat([train_feature_df, test_feature_df], axis=0, ignore_index=True)
features

,coupon rate,SP500 MD,Average daily 1-year SP500 return,Ratio to MA,US Corporate Bond Yield Spread,US Corporate Bond Yield Spread(3-5 year),US Corporate Bond Yield Spread(5-7 year),US Corporate Bond Yield Spread(7-10 year),US Corporate Bond Yield Spread(10+ year),US Generic Govt 3 Month Yield,...,event_type_subcategory_sum_Missing Coupon payment only,event_type_subcategory_sum_Missing Interest payment,event_type_subcategory_sum_Missing Loan payment,event_type_subcategory_sum_Missing Principal payment,event_type_subcategory_sum_Others,event_type_subcategory_sum_Pre-Negotiated Chapter 11,event_type_subcategory_sum_Protection,event_type_subcategory_sum_Receivership,event_type_subcategory_sum_Rehabilitation,event_type_subcategory_sum_Restructuring
0,7.500,-117.46020,-0.000189,125.407139,177.213028,134.012054,198.8153,191.364395,223.346344,0.1983,...,True,False,False,False,False,False,False,False,False,False
1,6.000,166.38276,0.000768,-4.603446,101.613617,77.032829,123.3998,105.932022,139.111115,0.0355,...,False,False,False,False,False,False,False,False,False,False
2,11.000,119.85752,0.000678,-11.950380,104.545959,77.416649,129.4317,111.818001,139.717407,0.0101,...,False,False,False,False,False,False,False,False,False,False
3,9.125,653.51208,0.001638,-2.494861,90.736633,64.654129,95.3731,92.141212,121.666237,0.0152,...,False,False,False,False,False,False,False,False,False,False
4,9.250,231.89472,0.000664,4.823413,98.533821,68.759308,93.4174,107.424469,139.741165,1.2865,...,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1720,3.250,147.33344,0.000556,3.538252,128.976395,95.360374,138.8445,138.946106,172.733887,0.0000,...,False,False,False,False,False,False,False,False,False,False
1721,10.000,175.31656,0.000554,-4.067337,121.170998,92.879501,123.2500,131.104904,162.916901,0.7516,...,False,True,False,False,False,False,False,False,False,False
1722,7.450,315.81748,0.000747,8.604100,126.786606,89.018188,114.9728,131.522430,171.701096,0.0864,...,False,False,False,False,False,False,False,False,False,False
1723,0.500,31.75120,-0.000157,-4.373852,126.595230,105.460007,121.6212,147.382416,163.772141,4.8375,...,False,False,False,False,False,False,False,False,False,False


In [16]:
labels = pd.concat([train_label_df, test_label_df], axis=0, ignore_index=True)
labels

,rr1_30
0,0.082481
1,0.378845
2,0.836149
3,0.987208
4,1.021458
...,...
1720,0.471411
1721,0.823750
1722,0.241612
1723,0.762054


In [29]:
# Define cross-validation setup
kf = KFold(n_splits=5, shuffle=True, random_state=42)
val_mae = []
val_mape = []
val_rmse = []
val_rsqr = []
oof_predictions = np.zeros(labels.shape[0])



for train_idx, val_idx in kf.split(features):
    all_predictions = []
    all_targets = []
    
    # Create training and validation datasets for the current fold
    X_train_fold, X_val_fold = features.iloc[train_idx], features.iloc[val_idx]
    y_train_fold, y_val_fold = labels.iloc[train_idx], labels.iloc[val_idx]
    
    # scaling features
    X_train_fold = scaler.fit_transform(X_train_fold)
    X_val_fold = scaler.transform(X_val_fold)
        
    # Initialize the model for this fold
    model = MLP(d_in=train_features.shape[1], d_layers=MODEL_CONFIG["linear"], dropout=MODEL_CONFIG["dropout"], d_out=1, activation_name=MODEL_CONFIG["activation_name"], negative_slope=MODEL_CONFIG['negative_slope'])
    model = nn.DataParallel(model, device_ids = DEVICE_LIST)
    model.to(DEVICE)
    
    # define optimizer
    optimizer = getattr(optim, MODEL_CONFIG["optimizer"]["optimizer"])(model.parameters(), **optim_config)
    
    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    
    # Prepare DataLoader for training
    train_dataset = CustomDataset(X_train_fold, y_train_fold.to_numpy())
    val_dataset = CustomDataset(X_val_fold, y_val_fold.to_numpy())
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)
    val_loader = DataLoader(val_dataset, batch_size=val_dataset.features.shape[0], shuffle=False)
        
    # Training of the model.
    model.train()
    for epoch in range(EPOCHS):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(DEVICE), target.to(DEVICE)

            optimizer.zero_grad()
            output = model(data)
            
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            # print(f'Epoch [{ep+1}], Train Loss: {train_loss:.4f}')

    # Validation of the model.
    model.eval()
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(val_loader):
            # data, target = data.to(DEVICE), target.to(DEVICE)
            if isinstance(model, nn.DataParallel):
                model = model.module  # Unwrap from DataParallel
            model = model.to('cpu')
            outputs = model(data)
            
            # Collect predictions and targets
        all_predictions.extend(outputs.cpu().numpy())
        all_targets.extend(target.cpu().numpy())

    # Convert to numpy arrays
    y_test_pred = np.array(all_predictions)
    y_test_true = np.array(all_targets)
    oof_predictions[val_idx] = y_test_pred

    # save metrics
    mae, mape, rmse, rsqr = calculate_metric(y_test_pred, y_test_true)
    val_mae.append(mae)
    val_mape.append(mape)
    val_rmse.append(rmse)
    val_rsqr.append(rsqr)
      

/home/nguyenhoa/miniconda3/envs/.henv/lib/python3.12/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/nguyenhoa/miniconda3/envs/.henv/lib/python3.12/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/nguyenhoa/miniconda3/envs/.henv/lib/python3.12/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broa

In [30]:
print(f"Test average mean absolute error: {statistics.mean(val_mae)}")
print(f"Test average mean absolute percentage error: {statistics.mean(val_mape)}")
print(f"Test average root mean squared error: {statistics.mean(val_rmse)}")
print(f"Test average R2: {statistics.mean(val_rsqr)}")

Test average mean absolute error: 0.347619891166687
Test average mean absolute percentage error: 3588.8164806365967
Test average root mean squared error: 0.2669967007013232
Test average R2: 0.34573233127593994


In [33]:
oof_predictions_df = pd.DataFrame({"predictions": oof_predictions.ravel()})
with pd.ExcelWriter(OOF_PREDICTIONS_FILE, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    # Write the new DataFrame to a new sheet
    oof_predictions_df.to_excel(writer, sheet_name=SHEET_NAME, index=False)